### Data description

#### Foursquare

For the assigment I will use the foursquare places API.

Foursquare is a location-based social network (LBSN) that allows people to share their location with friends by ‘checking-in’ at a given place using their smartphone. The application can also access the location-based recommendations left by other users. Foursquare has tens of millions of users and a huge database containing information of venues and users. including detailed information on users preferences, tips, menus, photo's, categories etc.




#### Accessing Foursquare information

Foursquare data can be retrieved using its api (application program interface). Using api's we can define a request and get the results returned in a json format.

There are multiple ways of calling the api (endpoints) where each endpoint has different inputparameters and returns different kind of information. 

For this assignment I use the 'explore' endpoint which returns the venues within a certain radius of a location provided as inputparameters (lat, lon) in the API call. In this api we can (but don't have to) provide one or more categoryId's which ensures we only get returned venues of a certain type (category). More information on the explore endpoint api here: https://developer.foursquare.com/docs/api-reference/venues/explore/




First, we perform an explore-call to retrieve the schoolnames (**using categoyId's for high schools**) and location information (ie lat and long values) of high schools within a 30.000m radius from downtown Los Angelos' lattitude and longitude. 

The api will return 100 results as a maximum per call. Using the offset parameter we can do multiple calls to get the next 100 results until we have reached the total number of results.

This is how we retrieve the location data of schools using the ___explore end point___ of the places API

In [42]:
#load libraries
import pandas as pd
import numpy as np

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import json

In [43]:
#Los Angelos' Lattitude and longitude values using the website https://www.latlong.net
locationLat= 34.052235
locationLon= -118.243683

In [110]:
#function te retrieve schools given an location and radius
def getSchools(lat, long, radius):
    
    CLIENT_ID = 'W3GBMV2TBYXHDKZLEHAT1FISU2HS12CZBNMMAWRNTHEOWJYY' 
    CLIENT_SECRET = 'CYTFPE31SS30UG2XEIAS5DNLOWTABZLYRXCCW5AIE1JCB1NV' 
    VERSION = '20200401' # Foursquare API version
    LIMIT = 50000 
    
    categoryId = '4bf58dd8d48988d13d941735' #high schools
    offset = 0
    
    school_list=[]

    #first call
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}&sortByDistance=1&offset={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius, 
    categoryId,
    LIMIT,
    offset)
            
    # make the GET request
    totalNumberOfSchools = requests.get(url).json()["response"]['totalResults'] #get total results
    schools = requests.get(url).json()["response"]['groups'][0]['items'] #get venues of category High School
        
    # return only relevant information for each school
    school_list.append([(
    s['venue']['name'], 
    s['venue']['location']['lat'], 
    s['venue']['location']['lng']) for s in schools])
    
  
    #create dataframe of results
    schools_df = pd.DataFrame([item for school in school_list for item in school])
    schools_df.columns = ['Name', 
                  'Latitude', 
                  'Longitude']
    
    # Consecutive calls
    while schools_df.Name.count() < totalNumberOfSchools:
        offset += 100 #increase by 100 to get next 100 results
        school_list.clear() # start with empty list for each iteration
        
        #call api again now using offset
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}&sortByDistance=1&offset={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        long, 
        radius, 
        categoryId,
        LIMIT,
        offset)
            
        # Get results again
        #totalNumberOfSchools = requests.get(url).json()["response"]['totalResults'] #get total results
        schools = requests.get(url).json()["response"]['groups'][0]['items'] #get venues of category High School
        
        # return only relevant information for each school
        school_list.append([(
        s['venue']['name'], 
        s['venue']['location']['lat'], 
        s['venue']['location']['lng']) for s in schools])
    
        # store results in dataframe and append to previous results 
        schools_df_temp = pd.DataFrame([item for school in school_list for item in school])
        
        schools_df_temp.columns = ['Name', 
                              'Latitude', 
                              'Longitude']
      
        # Append to dataframe
        schools_df = schools_df.append(schools_df_temp,sort=False,ignore_index=True) 

    
        
    return(schools_df)

To get an idea of the (way the) data that is returned: below the result of just one item (ie venue)

Using the location information of Los Angelos we can now get the name and location data of schools within a 30 km radius using our function

In [111]:
# get schools using 30.000m radius
dfSchools = getSchools(locationLat,locationLon,30000)

#look at first rows
dfSchools.head()


,Name,Latitude,Longitude
0,Ramón C. Cortines School of Visual and Perform...,34.059485,-118.244042
1,Central Los Angeles High School #9 (VAPA),34.059660,-118.244048
2,EightyTwo,34.045447,-118.237441
3,USC Hybrid High,34.054469,-118.254695
4,Downtown Business Magnet High School,34.063219,-118.250570


In [112]:
# Look at the total number of schools
dfSchools.Name.count()

218

Next, in a similar way a function will be created that determines per school (by repeatedly calling the api using the schools lat and long values from the previous step) if there are one of more venues of one of the following categories:
* Bar
* Liquor Store 
* Fast Food Restaurant

Only those venues will be selected where on of the above categories is the primary category of the venue.

A full list of available categories can be found here: https://developer.foursquare.com/docs/build-with-foursquare/categories/

*Please note not all categories are available for all countries*

Based on the counts per venue clusters will be formed using the kmeans algoritm. 
Finally the schools will be plot on a map by assigning each cluster its own color.